In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import os
import pandas as pd
from PIL import Image
from torchvision.transforms import ToTensor
from sklearn.model_selection import train_test_split
import numpy as np

device = torch.device('cuda:0')
class MultiLabelCNN(nn.Module):
    def __init__(self, num_labels, size):
        super(MultiLabelCNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc1 = nn.Sequential(
            nn.Linear(128 * (size[0]//8) * (size[1] // 8), 256),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        #分类
        self.sex = nn.Linear(256, 2)
        self.nation = nn.Linear(256, 53)
        self.right_left = nn.Linear(256, 2)
        self.who = nn.Linear(256,num_labels)
        #回归
        self.age = nn.Linear(256, 1)
        self.high = nn.Linear(256, 1)
        self.weight = nn.Linear(256, 1)
        

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        #print(x.shape)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        #print(f'x:{x,x.shape},sex:{self.sex(x),self.sex(x).shape}')
        sex = torch.softmax(self.sex(x), dim=1) 
        nation = torch.softmax(self.nation(x), dim=1)
        right_left = torch.softmax(self.right_left(x), dim=1)
        who = torch.softmax(self.who(x), dim=1)
        #回归
        age = self.age(x)
        high = self.high(x)
        weight = self.weight(x)
        return sex, age,high, weight, nation, right_left, who

/Users/fariy/opt/anaconda3/envs/deeplearning/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/fariy/opt/anaconda3/envs/deeplearning/lib/python3.7/site-packages/torchvision/image.so, 6): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: /Users/fariy/opt/anaconda3/envs/deeplearning/lib/python3.7/site-packages/torchvision/image.so
  Reason: Incompatible library version: image.so requires version 15.0.0 or later, but libjpeg.9.dylib provides version 12.0.0
  warn(f"Failed to load image Python extension: {e}")


In [2]:
from torchvision.transforms import transforms
# 加载模型
def load_model(model_class,size,numble_class, path='model.pth'):
    model = model_class(numble_class,size)
    model.load_state_dict(torch.load(path))
    model.eval()
    return model

# 预测函数
def predict(image_path, model, size=(96, 160)):
    # 图片预处理
    preprocess = transforms.Compose([
        transforms.Resize(size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    img = Image.open(image_path)
    img_tensor = preprocess(img)
    
    # 添加batch维度并在GPU上运行（如果可用）
    img_tensor.unsqueeze_(0)
    img_tensor = img_tensor.to(device=device)
    
    # 进行预测并返回结果
    with torch.no_grad():
        output_tuple = model(img_tensor)
        results = []
        for tensor in output_tuple:
            if tensor.numel() == 1:
                results.append(tensor.item())
            else:
                results.append(tensor.cpu().numpy())
                
        return tuple(results)

In [3]:
def max_prob_index(tensor,dict):
        # 转换为NumPy数组
    prob_array = np.array(tensor)

    # 找到最大值的索引
    max_index = np.argmax(prob_array)
    result = dict[max_index]
    return result

In [4]:
sex_dict = {0:'男', 1:'女'}
left_right = {0:'右利手', 1:'左利手'}
nation_dict = {0: '汉族', 1: '蒙古族', 2: '回族', 3: '藏族', 4: '维吾尔族', 5: '苗族', 6: '彝族', 7: '壮族', 8: '布依族', 9: '朝鲜族', 10: '满族', 11: '侗族', 12: '瑶族', 13: '白族', 14: '土家族', 15: '哈尼族', 16: '哈萨克族', 17: '傣族', 18: '黎族', 19: '傈僳族', 20: '佤族', 21: '畲族', 22: '高山族', 23: '拉祜族', 24: '水族', 25: '东乡族', 26: '纳西族', 27: '景颇族', 28: '柯尔克孜族', 29: '土族', 30: '达斡尔族', 31: '仫佬族', 32: '羌族', 33: '布朗族', 34: '撒拉族', 35: '毛难族', 36: '仡佬族', 37: '锡伯族', 38: '阿昌族', 39: '普米族', 40: '塔吉克族', 41: '怒族', 42: '乌孜别克族', 43: '俄罗斯族', 44: '鄂温克族', 45: '崩龙族', 46: '保安族', 47: '裕固族', 48: '京族', 49: '塔塔尔族', 50: '独龙族', 51: '鄂伦春族', 52: '赫哲族', 53: '门巴族', 54: '珞巴族', 55: '基诺族'}

In [5]:

num_labels = 31  # 身高，体重，年龄，性别, 民族, 左右利手
size = [96,160]
image_path = "数据集+信息表+说明/身份识别任务数据集/身份识别任务数据集/训练集/00024/2023-03-20-1.jpg"

model = load_model(MultiLabelCNN,size,num_labels, 'model/cnn_model.pth')
result = predict(image_path, model.to(device))
#print(result)

#print(f'年龄为：{sex_dict[torch.max(result[0])]} 岁')
print(f'年龄为：{result[1]} 岁')
print(f'身高为：{result[2]} cm')
print(f'体重为：{result[3]} kg')


print(f'性别为：{max_prob_index(result[0],dict=sex_dict)} 性')
print(f'民族为：{max_prob_index(result[4],dict=nation_dict)}')
print(f'左右利手为：{max_prob_index(result[5],dict=left_right)}')

FileNotFoundError: [Errno 2] No such file or directory: 'model/cnn_model.pth'

In [17]:
print(max_prob_index(result[0],dict=sex_dict))

女


In [26]:
print(max_prob_index(result[4],dict=nation_dict))

汉族


In [23]:
import json

json_string = '''
{
  "data": [
    {
      "id": "01",
      "name": "汉族"
    },
    {
      "id": "02",
      "name": "蒙古族"
    },
    {
      "id": "03",
      "name": "回族"
    },
    {
      "id": "04",
      "name": "藏族"
    },
    {
      "id": "05",
      "name": "维吾尔族"
    },
    {
      "id": "06",
      "name": "苗族"
    },
    {
      "id": "07",
      "name": "彝族"
    },
    {
      "id": "08",
      "name": "壮族"
    },
    {
      "id": "09",
      "name": "布依族"
    },
    {
      "id": "10",
      "name": "朝鲜族"
    },
    {
      "id": "11",
      "name": "满族"
    },
    {
      "id": "12",
      "name": "侗族"
    },
    {
      "id": "13",
      "name": "瑶族"
    },
    {
      "id": "14",
      "name": "白族"
    },
    {
      "id": "15",
      "name": "土家族"
    },
    {
      "id": "16",
      "name": "哈尼族"
    },
    {
      "id": "17",
      "name": "哈萨克族"
    },
    {
      "id": "18",
      "name": "傣族"
    },
    {
      "id": "19",
      "name": "黎族"
    },
    {
      "id": "20",
      "name": "傈僳族"
    },
    {
      "id": "21",
      "name": "佤族"
    },
    {
      "id": "22",
      "name": "畲族"
    },
    {
      "id": "23",
      "name": "高山族"
    },
    {
      "id": "24",
      "name": "拉祜族"
    },
    {
      "id": "25",
      "name": "水族"
    },
    {
      "id": "26",
      "name": "东乡族"
    },
    {
      "id": "27",
      "name": "纳西族"
    },
    {
      "id": "28",
      "name": "景颇族"
    },
    {
      "id": "29",
      "name": "柯尔克孜族"
    },
    {
      "id": "30",
      "name": "土族"
    },
    {
      "id": "31",
      "name": "达斡尔族"
    },
    {
      "id": "32",
      "name": "仫佬族"
    },
    {
      "id": "33",
      "name": "羌族"
    },
    {
      "id": "34",
      "name": "布朗族"
    },
    {
      "id": "35",
      "name": "撒拉族"
    },
    {
      "id": "36",
      "name": "毛难族"
    },
    {
      "id": "37",
      "name": "仡佬族"
    },
    {
      "id": "38",
      "name": "锡伯族"
    },
    {
      "id": "39",
      "name": "阿昌族"
    },
    {
      "id": "40",
      "name": "普米族"
    },
    {
      "id": "41",
      "name": "塔吉克族"
    },
    {
      "id": "42",
      "name": "怒族"
    },
    {
      "id": "43",
      "name": "乌孜别克族"
    },
    {
      "id": "44",
      "name": "俄罗斯族"
    },
    {
      "id": "45",
      "name": "鄂温克族"
    },
    {
      "id": "46",
      "name": "崩龙族"
    },
    {
      "id": "47",
      "name": "保安族"
    },
    {
      "id": "48",
      "name": "裕固族"
    },
    {
      "id": "49",
      "name": "京族"
    },
    {
      "id": "50",
      "name": "塔塔尔族"
    },
    {
      "id": "51",
      "name": "独龙族"
    },
    {
      "id": "52",
      "name": "鄂伦春族"
    },
    {
      "id": "53",
      "name": "赫哲族"
    },
    {
      "id": "54",
      "name": "门巴族"
    },
    {
      "id": "55",
      "name": "珞巴族"
    },
    {
      "id": "56",
      "name": "基诺族"
    }
  ]
}

'''

# 将JSON字符串转换为Python字典
json_data = json.loads(json_string)

# 提取"data"列表
data_list = json_data["data"]

# 创建一个新的字典，其中id作为键，name作为值
result_dict = {}
for item in data_list:
    result_dict[int(item["id"])-1] = item["name"]

print(result_dict)

{0: '汉族', 1: '蒙古族', 2: '回族', 3: '藏族', 4: '维吾尔族', 5: '苗族', 6: '彝族', 7: '壮族', 8: '布依族', 9: '朝鲜族', 10: '满族', 11: '侗族', 12: '瑶族', 13: '白族', 14: '土家族', 15: '哈尼族', 16: '哈萨克族', 17: '傣族', 18: '黎族', 19: '傈僳族', 20: '佤族', 21: '畲族', 22: '高山族', 23: '拉祜族', 24: '水族', 25: '东乡族', 26: '纳西族', 27: '景颇族', 28: '柯尔克孜族', 29: '土族', 30: '达斡尔族', 31: '仫佬族', 32: '羌族', 33: '布朗族', 34: '撒拉族', 35: '毛难族', 36: '仡佬族', 37: '锡伯族', 38: '阿昌族', 39: '普米族', 40: '塔吉克族', 41: '怒族', 42: '乌孜别克族', 43: '俄罗斯族', 44: '鄂温克族', 45: '崩龙族', 46: '保安族', 47: '裕固族', 48: '京族', 49: '塔塔尔族', 50: '独龙族', 51: '鄂伦春族', 52: '赫哲族', 53: '门巴族', 54: '珞巴族', 55: '基诺族'}
